# FAISS: 밀집 검색의 효과적인 구현을 위한 라이브러리
- FAISS는 대규모 데이터셋에서 고차원 벡터의 유사도 검색을 빠르게 수행할 수 있도록 설계된 라이브러리입니다.
- 도서관에서 책을 찾은 과정을 생각해봅시다.
- 책을 주제별로 분류하고 각 주제 안에서 다시 세부 카테고리로 나누어 체계적으로 관리합니다.
- FAISS도 이와 유사한 방식으로 작동합니다.
- 가장 기본적인 형태의 FAISS 인덱스(Flat 인덱스)는 모든 벡터를 있는 그대로 저장하고 전체 검색을 수행합니다.
- 이는 마치 모든 책을 한 줄로 늘어놓은 것과 같습니다. 작은 데이터셋에서는 이 방법이 정확하고 충분히 빠를 수 있지만, 데이터가 많아지면 검색속도가 급격히 느려집니다.

- FAISS는 더 발전된 인덱싱 방법을 제공합니다. 그 중 하나가 IVF(Inverted File) 인덱스를 활용하는 방식입니다.
- IVF인덱스는 벡터 공간을 여러 개의 클러스터로 나눕니다.
- 이는 마치 도서관에서 책을 주제별로 분류하는 것과 비슷합니다.
- 검색 시에는 질문 벡터와 가장 가까운 주제를 먼저 파악한 다음, 해당 주제의 책만을 살펴봄으로써 검색 속도를 크게 향상시킵니다.


- 예를들어 100만개의 문서 벡터가 있고, 이를 1000개의 클러스터로 나눴다고 가정해볼 경우
- 새로운 검색쿼리가 들어오면 먼저 가장 가까운 클러스터를 찾고, 클러스터내의 벡터들(약 1000개)만 유사도를 계산합니다.

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = "data/투자설명서.pdf"
loader = PyPDFLoader(file_path)

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = loader.load_and_split(doc_splitter)

- FAISS DB와 리트리버를 구축하는 단계로 넘어갑니다. 이 과정은 문서의 벡터화, 데이터베이스 생성 및 저장, 리트리버 생성의 세 부분으로 나눌 수 있습니다.


In [3]:
from langchain_ollama.embeddings import OllamaEmbeddings
embedding = OllamaEmbeddings(model="bge-m3")

- FAISS 데이터베이스를 생성하고 저장합니다.

In [4]:
# FAISS 라이브러리 임포트
from langchain_community.vectorstores import FAISS

# FAISS DB 생성 후 저장
faiss_store = FAISS.from_documents(docs, embedding)
faiss_store.save_local("data/DB")

- 저장된 FAISS DB를 다시 로드합니다.

In [7]:
# 저장된 DB 경로 지정 후, DB 로드
persist_directory = "data/DB"
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserialization=True)

- allow_dangerous_deserialization=True는 pickle 라이브러리(직렬화, 역직렬화 라이브러리)를 사용하여 DB를 불러오는 것입니다.

In [8]:
# FAISS 리트리버 생성
faiss_retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [10]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

# 관련 있는 문서 수집 후, 챗GPT로 최종 답변까지 수행
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.2),
    chain_type="stuff",
    retriever=faiss_retriever,
    return_source_documents=True, # 답변에 사용된 source_document 도 보여주도록 설정
)

In [11]:
qa_chain.invoke("이 회사가 발행한 주식의 총 발행량이 어느 정도야?")

{'query': '이 회사가 발행한 주식의 총 발행량이 어느 정도야?',
 'result': '이 회사가 발행한 주식의 총 발행량은 13,602,977주입니다.',
 'source_documents': [Document(id='600f79ef-e2da-4280-92b4-89ca39d9a9b1', metadata={'producer': 'iText® 5.5.9 ©2000-2015 iText Group NV (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2024-06-26T16:15:14+09:00', 'moddate': '2024-06-26T16:15:14+09:00', 'source': 'data/투자설명서.pdf', 'total_pages': 514, 'page': 328, 'page_label': '329'}, page_content='나. 자기 주식 \n \n당사는 본 보고서 작성기준일 현재 해당 사항이 없습니다. \n \n다. 다양한 종류의 주식 \n \n당사는 본 보고서 작성기준일 현재 해당 사항이 없습니다. \n  \n5. 정관에 관한 사항 \n \n가. 정관 변경 이력 \nⅠ. 발행할 주식의 총수 75,000,000 25,000,000 100,000,000 -\nⅡ. 현재까지 발행한 주식의 총수 13,602,977 - 13,602,977 -\nⅢ. 현재까지 감소한 주식의 총수 - - - -\n1. 감자 - - - -\n2. 이익소각 - - - -\n3. 상환주식의 상\n환 - - - -\n4. 기타 - - - -\nⅣ. 발행주식의 총수 (Ⅱ-Ⅲ) 13,602,977 - 13,602,977 -\nⅤ. 자기주식수 - - - -\nⅥ. 유통주식수 (Ⅳ-Ⅴ) 13,602,977 - 13,602,977 -\n정관변\n경일 해당주총명 주요변경사항 변경이유\n2020.03\n.25\n제14기 정기주\n주총회\n제2조(목적)\n제17조(전환사채의 발\n행)\n제18조(신주인수권부사\n채의 발행)\n제19조(이익